In [90]:
import numpy as np 

In [115]:
np.set_printoptions(precision=3, suppress=True)

In [91]:
capacities = [
    80,
    70,
    40,
    50
]
print(capacities)

[80, 70, 40, 50]


In [92]:
outage_rates = [
    0.1, 
    0.2,
    0.1,
    0.2
]
print(outage_rates)

[0.1, 0.2, 0.1, 0.2]


In [93]:
status = [
    True,
    False,
    True,
    True
]

In [173]:
demand = 90

In [107]:
capacities_fix = [[capacities[i], 0] for i in range(len(status)) if status[i]]
probabilities_fix = [[1-rates[i], rates[i]] for i in range(len(status)) if status[i]]

capacities_fix, probabilities_fix

([[80, 0], [40, 0], [50, 0]], [[0.9, 0.1], [0.9, 0.1], [0.8, 0.2]])

In [109]:
capacities_combination = list(itertools.product(*capacities_fix))
probabilities_combinations = list(itertools.product(*probabilities_fix))

print(capacities_combination)
print(probabilities_combinations)

[(80, 40, 50), (80, 40, 0), (80, 0, 50), (80, 0, 0), (0, 40, 50), (0, 40, 0), (0, 0, 50), (0, 0, 0)]
[(0.9, 0.9, 0.8), (0.9, 0.9, 0.2), (0.9, 0.1, 0.8), (0.9, 0.1, 0.2), (0.1, 0.9, 0.8), (0.1, 0.9, 0.2), (0.1, 0.1, 0.8), (0.1, 0.1, 0.2)]


In [170]:
capacity_merged = np.array([sum(i) for i in capacities_combination])
probability_merged = np.array([np.prod(i) for i in probabilities_combinations])

table = np.vstack([capacity_merged, probability_merged]).T
table = table[(-table[:, 0]).argsort(),:]
table

array([[170.   ,   0.648],
       [130.   ,   0.072],
       [120.   ,   0.162],
       [ 90.   ,   0.072],
       [ 80.   ,   0.018],
       [ 50.   ,   0.008],
       [ 40.   ,   0.018],
       [  0.   ,   0.002]])

In [172]:
cumulative_probability = np.zeros(len(probability_merged))

val = 0
for i in reversed(range(len(cumulative_probability))):
    val += table[i,1]
    cumulative_probability[i] = val

table = np.column_stack([table, cumulative_probability])
table

array([[170.   ,   0.648,   1.   ],
       [130.   ,   0.072,   0.352],
       [120.   ,   0.162,   0.28 ],
       [ 90.   ,   0.072,   0.118],
       [ 80.   ,   0.018,   0.046],
       [ 50.   ,   0.008,   0.028],
       [ 40.   ,   0.018,   0.02 ],
       [  0.   ,   0.002,   0.002]])

In [181]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    idx = np.argmax(capacity < demand)
    return cumulative_probability[idx]

In [186]:
lolp = get_lolp(table[:,0], table[:,2], demand)
lolp

0.046000000000000006